# All Results


In [6]:
import numpy as np

import umap

import torch
import json
import os
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from sklearn.decomposition import PCA

/home/user/understanding_understanding/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
def get_layer_activations(internal_reps, layer_idx):
    """
    Get the activations of a specific layer for all problems and all examples.

    internal_reps: list[tuple[tensor]] where internal_reps[question_num][layer_num]
     is a tensor of shape [batch=1, num_tokens, d_model=4096]

    We will return a tensor of shape [num_questions, num_tokens, d_model=4096]
    for the given layer
    """
    return torch.cat([internal_reps[i][layer_idx] for i in range(len(internal_reps))])


## x + y UMAP 20240709

```bash
python3 compute_minute_math_reps.py \
    --output_dir results/xyUMAP20240709 \
    --problem_type xy
```

In [8]:
# Define the directory containing the results
results_dir = 'xyUMAP20240709'

In [9]:
# Load the answers and problems files
answers = np.load(os.path.join(results_dir, 'answers.npy'))
problems = np.load(os.path.join(results_dir, 'problems.npy'))

# Load the internal representations and logits
internal_reps = torch.load(os.path.join(results_dir, 'internal_reps.pt'))
logits = torch.load(os.path.join(results_dir, 'logits.pt'))

# Load the arguments
with open(os.path.join(results_dir, 'args.json'), 'r') as f:
    args = json.load(f)

# Display the shapes and types of the loaded data
print("Answers shape:", answers.shape)
print("Problems shape:", problems.shape)
print("Internal Reps type:", type(internal_reps))
print("Internal Reps length:", len(internal_reps))
print("Logits type:", type(logits))
print("Logits length:", len(logits))

# Let's inspect the internal representations and logits a bit more closely
print("Example internal representation shape:", internal_reps[0][0][0].shape)
print("Example logits shape:", logits[0].shape)

# Display some examples from the loaded data
print("\nExample answers:", answers[0])
print("\nExample problems:", problems[0])
print("\nExample internal representation:", internal_reps[0][0][0])
print("\nExample logits:", logits[0])



/home/user/understanding_understanding/venv/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Answers shape: (100, 1)
Problems shape: (100, 17)
Internal Reps type: <class 'list'>
Internal Reps length: 100
Logits type: <class 'torch.Tensor'>
Logits length: 100
Example internal representation shape: torch.Size([17, 4096])
Example logits shape: torch.Size([17, 128256])

Example answers: [868]

Example problems: [128000     87    284    220    975     11    379    284    220     16
     26   9093    865    489    379    284    220]

Example internal representation: tensor([[-8.2970e-05,  2.5749e-04, -2.4605e-04,  ..., -3.2425e-04,
         -2.1553e-04,  4.7112e-04],
        [-2.0752e-03, -1.4038e-03,  6.1035e-03,  ...,  2.2888e-03,
          6.1951e-03,  1.1414e-02],
        [ 1.0376e-03, -6.8054e-03,  6.2943e-04,  ...,  2.5482e-03,
         -8.3618e-03, -8.8501e-03],
        ...,
        [ 3.3569e-03, -3.3760e-04,  2.4719e-03,  ..., -8.3008e-03,
          3.2654e-03, -7.3242e-03],
        [ 1.0376e-03, -6.8054e-03,  6.2943e-04,  ...,  2.5482e-03,
         -8.3618e-03, -8.8501e-03]

### Check correctness of model predictions

In [10]:
# compute the actual token-wise answers
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(args['model_name'])

# find the argmax over the logits for each 
# problem to get the predicted class

# logits has shape [num_questions, num_tokens, vocab_size]

final_logits = logits[:, -1, :]

# take the argmax over the final dim
predicted_class = torch.argmax(final_logits, dim=1)

# decode each prediction individually
predicted_class_str = [tokenizer.decode(i) for i in predicted_class]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
str_answers = [tokenizer.decode(i) for i in answers[:, 0]]

str_problems = []
for i in range(problems.shape[0]):
    str_problems.append(tokenizer.decode(problems[i, :]).replace('\n', ' '))

num_correct = 0
for i in range(len(str_answers)):
    if str_answers[i] == predicted_class_str[i]:
        num_correct += 1

num_correct / len(str_answers)

1.0

In [12]:
str_problems

['<|begin_of_text|>x = 14, y = 1; therefore x + y = ',
 '<|begin_of_text|>x = 7, y = 9; therefore x + y = ',
 '<|begin_of_text|>x = 7, y = 10; therefore x + y = ',
 '<|begin_of_text|>x = 7, y = 11; therefore x + y = ',
 '<|begin_of_text|>x = 13, y = 6; therefore x + y = ',
 '<|begin_of_text|>x = 13, y = 7; therefore x + y = ',
 '<|begin_of_text|>x = 8, y = 13; therefore x + y = ',
 '<|begin_of_text|>x = 8, y = 14; therefore x + y = ',
 '<|begin_of_text|>x = 6, y = 17; therefore x + y = ',
 '<|begin_of_text|>x = 4, y = 20; therefore x + y = ',
 '<|begin_of_text|>x = 6, y = 19; therefore x + y = ',
 '<|begin_of_text|>x = 4, y = 11; therefore x + y = ',
 '<|begin_of_text|>x = 7, y = 9; therefore x + y = ',
 '<|begin_of_text|>x = 7, y = 10; therefore x + y = ',
 '<|begin_of_text|>x = 6, y = 12; therefore x + y = ',
 '<|begin_of_text|>x = 6, y = 13; therefore x + y = ',
 '<|begin_of_text|>x = 20, y = 0; therefore x + y = ',
 '<|begin_of_text|>x = 1, y = 20; therefore x + y = ',
 '<|begin_of

In [13]:
# make a set of integer-valued answers
int_answers = [int(a) for a in str_answers]

In [14]:
class_ids = torch.tensor(answers[:, 0])
class_ids.shape

torch.Size([100])

In [ ]:

def get_umap(layer_reps, class_ids):
    """ Compute the UMAP of layer_reps, which is a tensor of shape [num_examples, num_tokens, token_dim]
    """
    print("Layer reps shape:", layer_reps.shape)
    print("Layer reps has type:", type(layer_reps))

    class_ids = class_ids.cpu()
    layer_reps = layer_reps.cpu()
    
    # Convert to numpy if it's a torch tensor
    if torch.is_tensor(layer_reps):
        layer_reps = layer_reps.detach().cpu().numpy()
    
    # Flatten the last two dimensions
    num_examples, num_tokens, token_dim = layer_reps.shape
    flattened_reps = layer_reps.reshape(num_examples, num_tokens * token_dim)
    
    # Apply UMAP
    reducer = umap.UMAP(random_state=42)
    umap_embedding = reducer.fit_transform(flattened_reps)
    # print("Output of reduce transform: ", umap_embedding)
    
    return umap_embedding

def get_class_relevance_pca(layer_reps, class_ids):
    """ Compute the PCA of layer_reps, which is a tensor of shape [num_examples, num_tokens, token_dim]
    We are going to compute the PCA which maximizes the variance of class means
    """
    flattened_reps = layer_reps.reshape(layer_reps.shape[0], -1).cpu()
    
    # Calculate class means efficiently
    unique_classes = np.unique(class_ids)
    print("flattened_reps type: ", type(flattened_reps))
    print("flattened_reps device: ", flattened_reps.device)
    print("class_ids type: ", type(class_ids))
    print("class_ids device: ", class_ids.device)
    class_means = np.array([flattened_reps[class_ids == c].mean(axis=0) for c in unique_classes])
    
    # Run PCA on the class means
    pca = PCA()
    pca.fit(class_means)
    
    # Project the original data onto the PCA components
    projected_data = pca.transform(flattened_reps)
    
    return projected_data




In [28]:
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

dim_reduction_func = get_class_relevance_pca


# iterate through each layer
num_layers = 33
purity_by_layer = []
for layer_num in range(num_layers):
    layer_activations = get_layer_activations(internal_reps, layer_num)
    low_dim_embedding = dim_reduction_func(layer_activations, class_ids)
    
    # Create Plotly figure
    fig = make_subplots(rows=1, cols=1)
    
    # Add scatter plot
    scatter = go.Scatter(
        x=low_dim_embedding[:, 0],
        y=low_dim_embedding[:, 1],
        mode='markers',
        marker=dict(
            size=8,
            color=int_answers,
            colorscale='Viridis',
            showscale=True
        ),
        text=[f"Problem: {prob}<br>Answer: {ans}<br>Predicted: {pred}" 
              for prob, ans, pred in zip(str_problems, str_answers, predicted_class_str)],
        hoverinfo='text'
    )
    
    fig.add_trace(scatter)
    
    # Update layout
    fig.update_layout(
        title=f'{dim_reduction_func.__name__} for Layer {layer_num}',
        xaxis_title='Dim 1',
        yaxis_title='Dim 2',
        width=1000,
        height=800
    )
    
    # Save as interactive HTML
    pio.write_html(fig, file=f'{results_dir}/{dim_reduction_func.__name__}_layer_{layer_num}.html')
    
    # Save as static PNG
    pio.write_image(fig, file=f'{results_dir}/{dim_reduction_func.__name__}_layer_{layer_num}.png')
    
    # Clear the figure to free up memory
    fig.data = []
    fig.layout = {}

flattened_reps type:  <class 'torch.Tensor'>
flattened_reps device:  cuda:0
class_ids type:  <class 'torch.Tensor'>
class_ids device:  cpu


TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [26]:
# # iterate through each layer
# num_layers = 33
# purity_by_layer = []
# for layer_num in range(num_layers):
#     layer_activations = get_layer_activations(internal_reps, layer_num)
#     umap_embedding = get_umap(layer_activations)
    
#     # Plot UMAP
#     plt.figure(figsize=(10, 8))
#     scatter = plt.scatter(umap_embedding[:, 0], umap_embedding[:, 1], c=int_answers, cmap='viridis')
#     plt.colorbar(scatter)
#     plt.title(f'UMAP for Layer {layer_num}')
#     plt.savefig(f'{results_dir}/umap_layer_{layer_num}.png')
#     plt.close()
    

In [ ]:
type(answers)

numpy.ndarray